In [413]:
import requests
import numpy as np
from bs4 import BeautifulSoup as BS
import pandas as pd
import re

In [550]:
#URL = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N.'
#URL = "https://www.opensecrets.org/races/candidates?cycle=2020&id=TN01&spec=N"
URL = "https://www.opensecrets.org/races/candidates?cycle=2020&id=TN02&spec=N"
#URL = "https://www.opensecrets.org/races/candidates?cycle=2020&id=TN08&spec=N"
page = requests.get(URL)
soup = BS(page.content, "html.parser")
#print(soup.prettify())

#replace tab, newline
def cleanUpString(param): 
    retValue = ""
    retValue = (param.replace('\t','')
                .replace('\n','')
                .replace('(','') 
                .replace(')','') 
                .strip()
               )
    #print ("Split")
    return(retValue)

In [449]:
# Declare the States
states = []

# Load Districts
districts = []
districts.append("https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N")


In [416]:
for idx, x in enumerate(districts):
    print(idx, x)

0 https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N


In [418]:
# find all districts page links in a state
#soup.find_all("a", class_="SubNav-link")[0]["href"]
for idx, x in enumerate(soup.find_all("a", class_="SubNav-link")):
    print(idx, x["href"])




0 /races/summary?cycle=2020&id=TNS2&spec=N
1 /races/summary?cycle=2020&id=TN01&spec=N
2 /races/summary?cycle=2020&id=TN02&spec=N
3 /races/summary?cycle=2020&id=TN03&spec=N
4 /races/summary?cycle=2020&id=TN04&spec=N
5 /races/summary?cycle=2020&id=TN05&spec=N
6 /races/summary?cycle=2020&id=TN06&spec=N
7 /races/summary?cycle=2020&id=TN07&spec=N
8 /races/summary?cycle=2020&id=TN08&spec=N
9 /races/summary?cycle=2020&id=TN09&spec=N


In [428]:
#soup.find_all("div", class_="Members--list-item")
#soup.find('div', attrs={'class' : 'Members--list-item'})

In [561]:
#Create Empty Dataframe for above columns
df_candidate = pd.DataFrame(columns = ['Name', 
                             'Party', 
                             'State', 
                             "DistrictNumber",
                             "Incumbment",
                             "WonRace",
                             "votePercentage",
                             "AmountRaised",
                             "AmountSpent"
                            ])
#print( df_candidate)
#Declare and initialize variable with defaults

candidateName=""
candidateParty=""
state = ""
districtNumber =""
incumbment = ""
wonRace = ""
votePercentage = np.NaN
amountRaised = np.NaN
amountSpent = np.NaN

candidateString = ""
title = soup.find_all('title', limit=1)[0].text.split(" ")
state = title[0]
districtNumber = title[2]
#print(title)

for idx, stateMembers in enumerate(soup.find_all("div", class_="Members--list-item")):        
    #votePercentage = stateMembers.find_all("span", class_="Members--vote-pct")[0].text.replace("(","").replace(")","")
    bio = stateMembers.find('strong').text
    #print (bio)      
    
    #candidateName = bio.split("(")[0]
    candidateNameSplit = bio.split("(", 1) # Max 1 occurence
    candidateName = cleanUpString(str(candidateNameSplit[0])) if len( candidateNameSplit) > 1 else ""
    
    votePercentageSplit = bio.rsplit("(",1)    
    votePercentage = cleanUpString(str(votePercentageSplit[1])) if len( votePercentageSplit) > 1 else ""
    votePercentage= votePercentage.split("%")[0]

    incumbment = str("Incumbent") if bio.find("Incumbent") > -1 else ""
    wonRace = str("Winner") if bio.find("Winner") > -1 else ""        

    if(re.findall("\(\w\)", bio)) :        
        candidateParty = cleanUpString(re.findall("\(\w\)", bio)[0])
    else :
        print("No Match")
        
    amountRaised = stateMembers.find_all("td", class_="Members--number")[0].text
    amountSpent = stateMembers.find_all("td", class_="Members--number")[1].text    
    # append rows to an empty DataFrame
    df_candidate = df_candidate.append({'Name' : candidateName, 'Party' : candidateParty, 'State' : state,
                   'DistrictNumber' : districtNumber, 'Incumbment' : incumbment, 'WonRace' :wonRace,
                   'votePercentage' : votePercentage, 'AmountRaised' :amountRaised, 'AmountSpent' : amountSpent},
                ignore_index = True)
    
df_candidate
  

    


C:\Users\kalea\AppData\Local\Temp\ipykernel_6084\1767727147.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_candidate = df_candidate.append({'Name' : candidateName, 'Party' : candidateParty, 'State' : state,
C:\Users\kalea\AppData\Local\Temp\ipykernel_6084\1767727147.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_candidate = df_candidate.append({'Name' : candidateName, 'Party' : candidateParty, 'State' : state,


,Name,Party,State,DistrictNumber,Incumbment,WonRace,votePercentage,AmountRaised,AmountSpent
0,Tim Burchett,R,Tennessee,02,Incumbent,Winner,67.7,"$1,336,276","$878,488"
1,Renee Hoyos,D,Tennessee,02,,,31.0,"$812,784","$816,793"
